In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations
import idaes
from itertools import permutations, product

from fim_doe import *

import pyomo.common.unittest as unittest

In [2]:
import cvxpy as cp

from reactor_kinetics import *

In [4]:
np.random.seed(271828) 
m = 2; n = 50
x = np.random.randn(m,n)

off = cp.Variable(boolean=True)

A = cp.Variable((2,2), PSD=True)
b = cp.Variable(2)
obj = cp.Maximize(cp.log_det(A))
constraints = [ cp.norm(A@x[:,i] + b) <= 1 + 20*off for i in range(n) ]
constraints += [cp.sum(off) <= 20]

prob = cp.Problem(obj, constraints)
optval = prob.solve(verbose=False) # seems to work, although it's not super accurate

print(f"Optimum value: {optval}")

NotImplementedError: 

## FIM reader

In [ ]:
def dataframe_return(data):
    fim1 = []
    fim1.append(data['P1'].to_list())
    fim1.append(data['P2'].to_list())
    fim1.append(data['P3'].to_list())
    fim1.append(data['P4'].to_list())
    
    return fim1 

In [ ]:
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

#t_measure = [0.125]



def FIM_reader(rank='onebyone'):
    
    FIM_set = []
    
    if rank=='onebyone':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
    elif rank=='onealong':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
        for idx, i in enumerate(t_measure):
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
        for idx, i in enumerate(t_measure):
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
            
    return FIM_set
        
FIM_set = FIM_reader(rank='onealong')
print(np.shape(FIM_set))
print(FIM_set[0][1][1])

In [ ]:
TotalF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        TotalF[i,j] = sum(FIM_set[k][i][j] for k in range(27))
        
print(TotalF)

print(np.trace(TotalF))
print(np.linalg.det(TotalF))
print(np.linalg.eigvals(TotalF))

## CVXPY - with variance

In [ ]:
no_res = 27

In [ ]:
def eval_fim(yi):
     
    fim = sum(yy*FIM_set[idx] for idx, yy in enumerate(yi))
    #fim = cp.bmat(FIM_set)
        
    return fim 

def a_opt(yi):
    fim = eval_fim(yi)
    #return cp.matrix_frac(np.identity(fim.shape==[0]), fim)
    return cp.trace(fim)
    
def d_opt(yi):
    fim = eval_fim(yi)
    return cp.log_det(fim)

def e_opt(yi):
    fim = eval_fim(yi)
    return cp.lambda_min(fim)

In [ ]:
yi = cp.Variable(no_res, integer=True)

p_cons = [cp.sum(yi) <= 20]

for i in yi:
    p_cons += [i<=1]
    p_cons += [i>=0]

obj = cp.Maximize(d_opt(yi))

problem = cp.Problem(obj, p_cons)

In [ ]:
yi = cp.Variable(no_res, boolean=True)

#constr1 = (expr1==yi)

constr1 = [cp.sum(yi) <= 20]

obj = cp.Maximize(d_opt(yi))

problem = cp.Problem(obj, constr1)


In [ ]:
print('Optimal value:', problem.solve(verbose=True))

print(yi.value)

In [ ]:
def eval_fim(ya, yb, yc):
     
    fim = sum(yy1*FIM_set[idx] for idx, yy1 in enumerate(ya))+sum(yy2*FIM_set[idx+9]*0.25 for idx, yy2 in enumerate(yb)) + sum(yy3*FIM_set[idx+18]*0.25 for idx, yy3 in enumerate(yc))
                
    return fim 

def a_opt(ya, yb, yc):
    fim = eval_fim(ya, yb, yc)
    #return cp.matrix_frac(np.identity(fim.shape==[0]), fim)
    return cp.trace(fim)
    
def d_opt(ya, yb, yc):
    fim = eval_fim(ya, yb, yc)
    return cp.log_det(fim)

def e_opt(ya, yb, yc):
    fim = eval_fim(ya, yb, yc)
    return -cp.lambda_min(fim)

In [ ]:
no_res_unit = 9

ya = cp.Variable(no_res_unit, integer=True)
yb = cp.Variable(no_res_unit, integer=True)
yc = cp.Variable(no_res_unit, integer=True)

p_cons = [cp.sum(ya*10) + cp.sum(yb*6) + cp.sum(yc*6) <= 80]

for i in ya:
    p_cons += [i<=1]
    p_cons += [i>=0]
for j in yb:
    p_cons += [j<=1]
    p_cons += [j>=0]
for d in yc:
    p_cons += [d<=1]
    p_cons += [d>=0]
    
obj = cp.Maximize(d_opt(ya, yb, yc))

problem = cp.Problem(obj, p_cons)



In [ ]:
print('First time: Optimal value:', problem.solve(solver=cp.GUROBI, verbose=True))

#print('Second time: Optimal value:', problem.solve(verbose=True, warm_start = True))

In [ ]:
print(ya.value)
print(yb.value)
print(yc.value)

In [ ]:
! echo $PATH

In [ ]:
import mosek

In [ ]:
print(cp.installed_solvers())